In [131]:
import pandas as pd
import numpy as np
import requests
import datetime


spacex_url="https://api.spacexdata.com/v4/launches/past"



In [132]:
response = requests.get(spacex_url)
data = pd.json_normalize(response.json())
data.head()
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

In [133]:
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

data['cores'] = data['cores'].map(lambda x:x[0])
data['payloads'] = data['payloads'].map(lambda x:x[0])

In [134]:
data['date_utc'] = data['date_utc'].astype('str')

In [135]:
data.dtypes

rocket           object
payloads         object
launchpad        object
cores            object
flight_number     int64
date_utc         object
dtype: object

In [136]:
data['date'] = data['date_utc'].str.slice(0,10).map(lambda x:x.strip())

In [137]:
data = data[pd.to_datetime(data['date']) <= pd.to_datetime(datetime.date(2020, 11, 13))]

In [138]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [139]:
BoosterVersion

[]

In [140]:
data['rocket']

0      5e9d0d95eda69955f709d1eb
1      5e9d0d95eda69955f709d1eb
3      5e9d0d95eda69955f709d1eb
4      5e9d0d95eda69955f709d1eb
5      5e9d0d95eda69973a809d1ec
                 ...           
101    5e9d0d95eda69973a809d1ec
102    5e9d0d95eda69973a809d1ec
103    5e9d0d95eda69973a809d1ec
104    5e9d0d95eda69973a809d1ec
105    5e9d0d95eda69973a809d1ec
Name: rocket, Length: 94, dtype: object

In [141]:
async def getBoosterVersion(d):
    for x in d['rocket']:
        response = await requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

In [142]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
async def getLaunchSite(d):
    for x in d['launchpad']:
        session = requests.Session()
        # session.headers.update({"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"})
        response = await requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x),headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0",}).json()
        Longitude.append(response['longitude'])
        Latitude.append(response['latitude'])
        LaunchSite.append(response['name'])

In [143]:
data['cores']

0      {'core': '5e9e289df35918033d3b2623', 'flight':...
1      {'core': '5e9e289ef35918416a3b2624', 'flight':...
3      {'core': '5e9e289ef3591855dc3b2626', 'flight':...
4      {'core': '5e9e289ef359184f103b2627', 'flight':...
5      {'core': '5e9e289ef359185f2b3b2628', 'flight':...
                             ...                        
101    {'core': '5ef670f10059c33cee4a826c', 'flight':...
102    {'core': '5e9e28a7f3591817f23b2663', 'flight':...
103    {'core': '5e9e28a6f35918c0803b265c', 'flight':...
104    {'core': '5ef670f10059c33cee4a826c', 'flight':...
105    {'core': '5f57c5440622a633027900a0', 'flight':...
Name: cores, Length: 94, dtype: object

In [151]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
async def getCoreData(d):
    for core in d['cores']:
            if core['core'] != None:
                response = await requests.get("https://api.spacexdata.com/v4/cores/"+core['core'],headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0",}).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [145]:
getBoosterVersion(data)

<coroutine object getBoosterVersion at 0x0000022CFBA997C0>

In [146]:
BoosterVersion[:5]

[]

In [147]:
getLaunchSite(data)
LaunchSite

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_452/3383825824.py:1: RuntimeWarning: coroutine 'getLaunchSite' was never awaited
  getLaunchSite(data)


[]

In [157]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
async def getPayloadData(data):
    for load in data['payloads']:
        response = await requests.get("https://api.spacexdata.com/v4/payloads/"+str(load),headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0",}).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [155]:
getCoreData(data)

<coroutine object getCoreData at 0x0000022CFB923840>

In [159]:
getPayloadData(data)
PayloadMass

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_452/563363736.py:1: RuntimeWarning: coroutine 'getPayloadData' was never awaited
  getPayloadData(data)


[]

In [160]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}
